In [1]:
import numpy as np
import sympy as sp
import qiskit
from qiskit import Aer, execute
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
import itertools as it

import os, sys
sys.path.append("../")
from quantum_circuit.sp_circuit import Circuit
from scipy.special import rel_entr

In [2]:
def prob_KL(num_qubits, num_gate):
    qc = Circuit(num_qubits)

    gate_list = ['h', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'rxx', 'ryy', 'rzz', 'swap', 'cx', 'cz']
    for i in range(num_gate):
        rnd_gate = np.random.choice(gate_list)
        if rnd_gate in ['h', 'x', 'y', 'z']:
            wire = np.random.randint(num_qubits)
            getattr(qc, rnd_gate)(wire)
        elif rnd_gate in ['rx', 'ry', 'rz']:
            theta = 2 * np.pi * np.random.rand()
            wire = np.random.randint(num_qubits)
            getattr(qc, rnd_gate)(theta, wire)
        elif rnd_gate in ['rxx', 'ryy', 'rzz']:
            theta = 2 * np.pi * np.random.rand()
            wire1, wire2 = np.random.choice(range(num_qubits), size=2, replace=False)
            getattr(qc, rnd_gate)(theta, wire1, wire2)
        elif rnd_gate in ['swap', 'cx', 'cz']:
            wire1, wire2 = np.random.choice(range(num_qubits), size=2, replace=False)
            getattr(qc, rnd_gate)(wire1, wire2)

    sv = Statevector.from_label(num_qubits * '0')
    sv = sv.evolve(qc.qiskit_circuit)
    sv = sv.reverse_qargs()
    qc.evolve_state()
    psi = np.array(sp.Abs(qc.final_state.evalf()), dtype=float).reshape(-1)
    qiskit_prob = np.around(np.array(sv.probabilities(), dtype=float), decimals=5)
    sympy_prob  = np.around(psi**2, decimals=5)
    return qiskit_prob, sympy_prob

In [3]:
for round in range(10):
    qiskit_prob, sympy_prob = prob_KL(num_qubits=3, num_gate=20)
    result_KL = np.sum(rel_entr(qiskit_prob, sympy_prob))
    if result_KL != 0:
        print(f"Round {round} failed, with KL = {result_KL}")
    else:
        print(f"Round {round} success!")

Round 0 success!
Round 1 success!
Round 2 success!
Round 3 success!
Round 4 success!
Round 5 success!
Round 6 success!
Round 7 success!
Round 8 success!
Round 9 success!
